# Pickling

In [1]:
import pickle
import os
from pathlib import Path
import pandas as pd

# Function to save a single dataframe using Pickle
def save_dataframe(dataframe, filename):
    """
    Arguments:
    - dataframe: The pandas DataFrame to save.
    - filename: The file path (including name) where the DataFrame will be stored.
    """
    with open(filename, 'wb') as file:
        pickle.dump(dataframe, file)  # Serialize 
        print(f"DataFrame saved to {filename}")

# Function to save multiple dataframes iteratively
def save_multiple_dataframes(dataframes, save_directory):
    """
    Arguments:
    - dataframes: A list of pandas DataFrames to save.
    - save_directory: Directory path to store the pickle files.
    """
    # Ensure the directory exists
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    
    # Iterate through the list of DataFrames
    for i, df in enumerate(dataframes):
        filename = os.path.join(save_directory, f"dataframe_{i + 1}.pkl")
        save_dataframe(df, filename)

# Function to load a single dataframe from a pickle file
def load_dataframe(filename):
    """
    Arguments:
    - filename: The file path (including name) to load the DataFrame from.
    Returns:
    - dataframe: The pandas DataFrame loaded from the pickle file.
    """
    with open(filename, 'rb') as file:
        dataframe = pickle.load(file)  # Deserialize the DataFrame from the pickle file
        print(f"DataFrame loaded from {filename}")
        return dataframe

# Function to load multiple dataframes from pickle files in a directory
def load_multiple_dataframes(save_directory):
    """
    Arguments:
    - save_directory: Directory path where pickle files are stored.
    Returns:
    - dataframes: A list of pandas DataFrames loaded from the pickle files.
    """
    dataframes = []
    pickle_files = [f for f in os.listdir(save_directory) if f.endswith('.pkl')]  # List of pickle files
    
    # Iterate over all pickle files and load them
    for pickle_file in pickle_files:
        file_path = os.path.join(save_directory, pickle_file)
        df = load_dataframe(file_path)
        dataframes.append(df)
    
    return dataframes


In [ ]:
#final version

import telemetry_parser
import pandas as pd
import numpy as np
from config import config
from pprint import pprint
from pathlib import Path

def extract_imu_data(file: Path):
    tp = telemetry_parser.Parser(str(file))
    # Define the columns for the dataframe
    kolommen = ['TIMESTAMP', 'ACCL_x', 'ACCL_y', 'ACCL_z', 'GYRO_x', 'GYRO_y', 'GYRO_z']
    imu_data_df = pd.DataFrame(columns=kolommen)

    # Retrieve the length of the data, assuming tp.normalized_imu() returns a list of IMU data samples
    imu_data = tp.normalized_imu()  # Data retrieved from normalized_imu function
    length = len(imu_data)  # Get the number of IMU data samples
    # print(length)

    all_rows = []

    # Iterate over each telemetry sample (second)
    for i in range(length):
        timestamp_s = imu_data[i]['timestamp_ms'] / 1000  # Convert from ms to seconds
        
        # Retrieve accelerometer and gyroscope data for the current sample
        accl_data = imu_data[i]['accl']
        gyro_data = imu_data[i]['gyro']
        
        # Extract x, y, z components from the accelerometer and gyroscope data
        accl_x, accl_y, accl_z = accl_data
        gyro_x, gyro_y, gyro_z = gyro_data
        
        # Create a new row with the timestamp and sensor data
        new_row = [timestamp_s, accl_x, accl_y, accl_z, gyro_x, gyro_y, gyro_z]
        all_rows.append(new_row)

    imu_data_df = pd.DataFrame(all_rows, columns=kolommen)

    # Optionally save the DataFrame to a CSV file
    imu_data_df.to_csv(config.DATA_DIR / 'normalized_imu_data.csv', index=False)
    
    return imu_data_df


x = (extract_imu_data('data/raf_frames.MP4'))
display(x.head(100))
display(x.tail(100))

testing

In [ ]:
df1 = extract_imu_data('data/raf_frames.MP4')
df2 = extract_imu_data('data/GH010045.MP4')
directory = 'pickled_bytestream'

dataframes = [df1, df2]

# Check if the directory exists, if not create it
if not os.path.exists(directory):
    os.makedirs(directory)

# Now, save the DataFrame to a pickle file inside the directory
filename = os.path.join(directory, 'imu_data.pkl')  # Specify the file name with .pkl extension

save_dataframe(df1, filename)

save_multiple_dataframes(dataframes, directory)

In [ ]:
# load_dataframe('pickled_bytestream/imu_data.pkl')

load_multiple_dataframes('pickled_bytestream')